In [32]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
doclist_txt[:5]

20-Jan-2017 17:17:13 INFO preprocessing: Creating document list from TXT files ...
20-Jan-2017 17:17:13 DEBUG preprocessing: 17 entries in document list.


['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

20-Jan-2017 17:17:13 INFO preprocessing: Creating document list from CSV files ...
20-Jan-2017 17:17:13 DEBUG preprocessing: 16 entries in document list.


['corpus_csv/Howard_GodsoftheNorth.txt.csv',
 'corpus_csv/Poe_EurekaAProsePoem.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Poe_ThePurloinedLetter.txt.csv',
 'corpus_csv/Howard_ShadowsintheMoonlight.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

20-Jan-2017 17:17:13 INFO preprocessing: Creating document labels ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Document labels available


['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [pre.tokenize(corpus_txt) for doc in corpus_txt]
doc_tokens[:1]

20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_EurekaAProsePoem.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_TheDevilinIron.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Lovecraft_TheShunnedHouse.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_SchadowsinZamboula.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_AStudyinScarlet.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_TheCaskofAmontillado.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_TheMasqueoftheRedDeath.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_GodsoftheNorth.txt ...
20-Jan-2017 17:17:13 DEBUG preprocessing: Accessing TXT document corpus_txt/Kipling_TheEndofthePassage.txt ...
20-Jan-2017 17:17:13 DEBUG

[<generator object tokenize at 0x7fa7180a3258>]

#### Create Doc-Term Matrix

In [24]:
id_types, doc_dict, doc_ids = pre.create_large_TF_matrix(doc_labels, doc_tokens)

termdoc_matrix = {v : k for k, v in id_types.items()}

id_docs = {v : k for k, v in doc_ids.items()}

In [ ]:
largecounter = pre.create_large_counter(doc_labels, doc_tokens, termdoc_matrix)

largecounter = {id_docs[key] : value for key, value in largecounter.items()}

In [26]:
len(largecounter)

In [ ]:
len(sparse_index)

In [ ]:
len(pd.DataFrame(np.zeros((len(sparse_index.get_level_values('token_id')),1), dtype = int), index = sparse_index))

In [ ]:
sparse_index = pre.create_sparse_index(largecounter)

In [ ]:
sparse_df = populate_two(sparse_index, largecounter)

In [ ]:
sparse_index = pre.create_sparse_index(largecounter)

sparse_df = pre.populate_two(sparse_index, largecounter)

##### Market Matrix Datei anlegen

In [ ]:
sparse_df.to_csv('sparse_drama.mm', sep = ' ')

In [ ]:
sparse_df = sparse_df.sortlevel()

In [ ]:
max(sparse_df[0])

In [ ]:
max(sparse_df.index.get_level_values("doc_id"))

In [ ]:
max(sparse_df.index.get_level_values("token_id"))

In [ ]:
sum(sparse_df[0])

##### Speichern der Market Matrix

In [ ]:
num_docs = max(sparse_df.index.get_level_values("doc_id"))
num_types = max(sparse_df.index.get_level_values("token_id"))
sum_counts = sum(sparse_df[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    #f.write(header_string)
    sparse_df.to_csv( f, sep = ' ', header = None)

##### Create Feature Remove Lists

In [ ]:
stopword_list = pre.find_stopwords(docterm_matrix, 100)
stopword_list

In [ ]:
hapax = set(docterm_matrix[docterm_matrix == 1].index)
#hapax = set(docterm_matrix[(docterm_matrix < 2).all(axis=1)].index)
len(hapax)

In [ ]:
hapax_list = collection.find_hapax(docterm_matrix)
len(hapax_list)

In [ ]:
feature_list = stopword_list.union(hapax_list)
print(feature_list)
clean_term_frequency = docterm_matrix.drop(feature_list)

##### Remove Features

In [ ]:
feature_list = stopword_list.union(hapax_list)
clean_term_frequency = collection.remove_features(docterm_matrix, feature_list)

##### Lemmata anhand POS-Tags auswählen

In [ ]:
lemmas = collection.filter_POS_tags(corpus_csv)
next(lemmas)

## Topic Modeling with Gensim

In [ ]:
corpusPath = "gb_plain.mm"

mm = MmCorpus(corpusPath) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen


In [ ]:
#in case you're only loading the corpus - build dict first

import pickle

pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [ ]:
id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [ ]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [ ]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [ ]:
model = LdaModel(corpus=mm, id2word=id_types, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen

In [ ]:
topic_nr_x = model.get_topic_terms(10)

[id_types[i[0]] for i in topic_nr_x]

In [ ]:
topics = model.show_topics(num_topics = 60)

In [ ]:
topics

## Topic Modeling mit Mallet

In [ ]:
path_to_corpus = "corpus_txt"

mallet = mal.call_mallet(path_to_corpus)

## Visualisierung 

In [ ]:
lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)

#### Heatmap

In [ ]:
heatmap = visual.make_heatmap()

In [ ]:
visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [ ]:
vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [ ]:
vis.make_interactive()

In [ ]:
vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

In [ ]:
![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)